In [93]:
from collections import OrderedDict
from sqlalchemy import create_engine
from Levenshtein import distance
import numpy as np
import pandas as pd
import numba as nb
import scipy as sp
import pymysql, time, re, math, pickle, h5py, os
from scipy.sparse import csr_matrix
from IPython.core.debugger import Tracer
# import warnings
# warnings.filterwarnings('ignore')

# Connecting to database

In [94]:
config = {
    'user': 'staging',
    'passwd': '$dsaGSD92&76',
    'host': '10.106.48.3',
    'port': '3306',
}

In [95]:
dbName = "partly_staging"
charSet = "utf8mb4"
cusrorType = pymysql.cursors.DictCursor

In [96]:
path = './Data/csr_matrix.h5'

In [97]:
hf = h5py.File(path, "r")
vdset = hf['vehicle_scores']
edset = hf['engine_scores']

In [98]:
engine_df = pd.DataFrame(data=hf.get('engine_scores')[:], columns=["supercheap_id", "top_mappings"])
vehicle_df = pd.DataFrame(data=hf.get('vehicle_scores')[:], columns=["supercheap_id", "top_mappings"])

In [99]:
def convert_dtype(df):
    df['supercheap_id'] = df['supercheap_id'].astype('int')
    df['top_mappings'] = df['top_mappings'].str.decode('utf-8')

In [100]:
convert_dtype(engine_df)
convert_dtype(vehicle_df)

# Writing to database

## Creating tables

In [101]:
try:
    # Connect to database
    dbc   = pymysql.connect(host=config['host'], user=config['user'], password=config['passwd'],
                                     db=dbName, charset=charSet,cursorclass=cusrorType)
    # Create a cursor object
    cursor = dbc.cursor()                                     
    # Creating mapping table for engine
    sqlQuery = """CREATE TABLE IF NOT EXISTS uvdb_supercheap_defined_engine(id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                supercheap_id bigint(20) unsigned,
                defined_engine_id int(10) unsigned,
                score real(5, 2),
                PRIMARY KEY (`id`),
                CONSTRAINT `uvdb_defined_engine_id_foreign` FOREIGN KEY (`defined_engine_id`) REFERENCES `uvdb_defined_engines` (`id`))
                """
    # Execute the sqlQuery
    cursor.execute(sqlQuery)
    # Creating mapping table for vehicle
    sqlQuery = """CREATE TABLE IF NOT EXISTS uvdb_supercheap_full_vehicle(id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                supercheap_id bigint(20) unsigned,
                full_vehicle_id int(10) unsigned,
                score real(5, 2),
                vin_code varchar(17) COLLATE utf8mb4_unicode_ci DEFAULT NULL,
                PRIMARY KEY (`id`),
                KEY `uvdb_supercheap_full_vehicle_vincode_index` (`vin_code`),
                CONSTRAINT `uvdb_full_vehicle_id_foreign` FOREIGN KEY (`full_vehicle_id`) REFERENCES `uvdb_full_vehicles` (`id`))
                """
    # Execute the sqlQuery
    cursor.execute(sqlQuery)
    #Fetch all the rows
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print("Exeception occured:{}".format(e))
finally:
    dbc.close()

# Adding VIN code to vehicle's mapping table

## Loading detail from supercheap data 2

In [102]:
def load_table(config):
    db_engine = create_engine(
    f"mysql+pymysql://{config['user']}:{config['passwd']}@{config['host']}:{config['port']}", pool_recycle=3600)
    dbc = db_engine.connect()

    sql = "select id, detail from ebay.supercheap_data_2"
    
    detail_df = pd.read_sql(sql, dbc)
    with open('detail_cache.dat', 'wb') as fh:
        pickle.dump(detail_df, fh)
    return detail_df

In [103]:
refresh = 0
if refresh == 1:
    detail_df = load_table(config)
else:
    with open('detail_cache.dat', 'rb') as fh:
        detail_df = pickle.load(fh)

## Getting VIN

In [104]:
def get_vin(row):
    debug = 0
    vin = ''
    m = re.search(' ([0-9A-Z#]{5,})( +\[| +,|$)', row.detail)
    if m:
        vin = m[1]
    elif debug == 1:
        print(f'id: {row.id} vin: {row.detail}')
    return pd.Series({'id': row.id, 'vin_code': vin})

In [105]:
refresh = 0
if refresh == 1:
    id_vin = detail_df.apply(get_vin, axis=1)
else:
    with open('vin_id_cache.dat', 'rb') as fh:
        id_vin = pickle.load(fh)

## Merging VIN to Super vehicle

In [106]:
vin_vehicle = vehicle_df.merge(id_vin, how='left', left_on="supercheap_id", right_on="id")
vin_vehicle.drop('id', axis=1, inplace=True)

## Inserting data

In [107]:
def form_insert_data(df, vin_mode=False):
    res = []
    for _, row in df.iterrows():
        m = re.search("^(\d+): ([0-9.]+)\t", row.top_mappings)
        vin = ''
        if vin_mode:
            new_row = [row.supercheap_id, m[1], m[2], row.vin_code]
        else:
            new_row = [row.supercheap_id, m[1], m[2]]

        res.append(tuple(new_row))
    return res

In [108]:
def insert_data(dbc, query, df, vin_mode=False):
    try:
        cursor = dbc.cursor()
        data = form_insert_data(df, vin_mode)
        cursor.executemany(query, data)
        cursor.close()
        dbc.commit()
    except Exception as e:
        print("Exeception occured:{}".format(e))
        dbc.rollback()
        dbc.close()

In [109]:
dbc = pymysql.connect(host=config['host'], user=config['user'], password=config['passwd'],
                        db=dbName, charset=charSet,cursorclass=cusrorType)
query_engine = '''INSERT INTO uvdb_supercheap_defined_engine(supercheap_id, defined_engine_id, score)
            VALUES(%s, %s, %s)'''
insert_data(dbc, query_engine, engine_df)
query_vehicle = '''INSERT INTO uvdb_supercheap_full_vehicle(supercheap_id, full_vehicle_id, score, vin_code)
            VALUES(%s, %s, %s, %s)'''
insert_data(dbc, query_vehicle, vin_vehicle, True)
dbc.close()